Transformer based LLMs are doing really well for themselves right now. Not only have they crushed most existing language benchmarks into the dust, but recent iterations have seen success in e.g groking high school math questions, writing papers for overworked undergraduates, and even convincing google engineers to retain lawyers on their behalf. Debates on the further scaling of these models often seems to point towards the dawn of AGI arriving just as soon as somebody stumps up the cash to wire enough GPUs in parallel. 

If this isn't enough, a slew of recent papers seem to have shown that LLMs actually predict human brain activity [@caucheteux2022deep; @schrimpf2021neural]. That's right! LLMS are so amazing, that we get human brain prediction for free. Despite never seeing a single blip of human brain activity, it looks like they reliably correlate with it. In this brief post I'll examine this claim along with some python code. 

The Schrimpf et al paper tests a variety of pretrained LLM models on neural datasets, finding openAI's gpt2 (whicb is the largest) to be the best. The authors follow a basic pipeline.

1. A dataset is chosen containing observed fmri values for participants as they listen to recorded text. The dataset is arranged so that each group of fmri values is matched with the sentence that was being listened to at that timepoint.

2. The sentences are passed into gpt2 (or a smaller LLM), and the LLM's activations are recorded at each layer.

3. Thus we have two datasets. One is a set of fmri values paired with sentences, and the other is a set of gpt2 values paired with the same sentences.

4. For each column of fmri values, and for each layer of gpt2, a Linear Regression model is fitted between gpt2's activations at that layer, and the column of fmri values. A train/test split is used.

5. Each of these models is evaluated on the pearson corrleation between the real and predicted fmri values for the holdout test data.

6. For each layer, the correlations are aggregated and then normalized via a noise ceiling to produce a 'brain score' for that layer.


Schrimpf et al were kind enough to make their code available [here](https://github.com/mschrimpf/neural-nlp). I had some trouble getting it installed on my machine[^1], but once its up and running the pipeline can be executed more or less like this:

```
from neural_nlp import score as score_function
from neural_nlp.models import model_pool

model='gpt2-xl'
benchmark = 'Blank2014fROI-encoding'
layers=None
subsample=None


(raw, normalized )= score_function(model=model,
            layers=layers,
            subsample=subsample,
            benchmark=benchmark)

print(raw)

----> <xarray.Score (layer: 49, neuroid: 60)>
array([[ 0.03816879,  0.03128891,  0.0259099 , ..., -0.04238258,
         0.00313762,  0.06058007],
       [ 0.04301853,  0.03194299,  0.02471395, ...,  0.00912335,
         0.00075296,  0.00516362],
       [ 0.04992471,  0.03059291,  0.04292314, ..., -0.01476794,
         0.0325259 , -0.00104979],
       ...,
       [ 0.00529838,  0.01703644,  0.07560671, ...,  0.02316549,
         0.0651233 ,  0.03626202],
       [ 0.0134888 ,  0.03466974,  0.0436985 , ...,  0.01856872,
         0.05222637,  0.01985603],
       [ 0.00987461,  0.02937339,  0.02830886, ...,  0.01802316,
         0.04671053,  0.02116557]])

print(normalized)

----> <xarray.Score (layer: 49, aggregation: 2)>
array([[0.05130709, 0.03568719],
       [0.02033876, 0.05590313],
       [0.12162405, 0.02070109],
       [0.16783761, 0.02983402],
       [0.20021165, 0.10040615],
       [0.17921272, 0.03084053],
       [0.10905765, 0.0314293 ],
       [0.10848611, 0.02249233],
       [0.11885668, 0.06864083],
       [0.08059633, 0.02379569],
       [0.11712025, 0.05120604],
       [0.06881359, 0.04037802],
       [0.09987826, 0.0295003 ],
       [0.16175823, 0.06477531],
       [0.17899809, 0.04018901],
       [0.17981657, 0.01172136],
       [0.20943076, 0.01155219],
       [0.1825894 , 0.03508501],
       [0.16672216, 0.05114289],
       [0.20879453, 0.09660362],
...
       [0.40728323, 0.11604758],
       [0.33939299, 0.0544715 ],
       [0.34116018, 0.09799635],
       [0.3864782 , 0.0375988 ],
       [0.37378828, 0.04645422],
       [0.30747895, 0.05700194],
       [0.2637388 , 0.07412165],
       [0.3210561 , 0.03897856],
       [0.27245259, 0.02425598],
       [0.26047741, 0.01911099],
       [0.2514971 , 0.01159271],
       [0.23486112, 0.02126413],
       [0.22253888, 0.02934557],
       [0.20027159, 0.01865831],
       [0.20314536, 0.00693946],
       [0.23661897, 0.04025427],
       [0.21830474, 0.02957045],
       [0.22526647, 0.01812639],
       [0.19112372, 0.01546557],
       [0.18473642, 0.00973036]])


print(list([np.round(x,4) for x in raw.mean(axis=1).values]))

--->[0.0063, 0.0063, 0.0237, 0.0249, 0.0378, 0.0342, 0.0228, 0.0241, 0.022, 0.0185, 0.0259,
 0.022, 0.0245, 0.0283, 0.03, 0.0409, 0.0424, 0.0373, 0.0422, 0.0425, 0.0447, 0.0374, 0.0437,
  0.0442, 0.0518, 0.0503, 0.0559, 0.0594, 0.0582, 0.0619, 0.0519, 0.0539, 0.0569, 0.0548, 0.0498,
   0.0468, 0.0504, 0.049, 0.0457, 0.0515, 0.0483, 0.0475, 0.0472, 0.0473, 0.0438, 0.041, 0.0407, 
   0.0385, 0.0366]

print([np.round(x,4) for x in normalized.values[:,0]])

--->[0.0513, 0.0203, 0.1216, 0.1678, 0.2002, 0.1792, 0.1091, 0.1085, 0.1189, 0.0806, 0.1171,
 0.0688, 0.0999, 0.1618, 0.179, 0.1798, 0.2094, 0.1826, 0.1667, 0.2088, 0.2309, 0.1709,
  0.206, 0.2154, 0.265, 0.2758, 0.3062, 0.2916, 0.367, 0.4073, 0.3394, 0.3412, 0.3865, 0.3738,
   0.3075, 0.2637, 0.3211, 0.2725, 0.2605, 0.2515, 0.2349, 0.2225, 0.2003, 0.2031,
    0.2366, 0.2183, 0.2253, 0.1911, 0.1847]

---->
```

Coming back in around a day or so (with an ok gpu) we get the above results. The output is two instances of `DataAssembly`, the first one giving the raw correlations, and the second providing the final scores, normalized by a noise ceiling (a correlation upper bound that the authors precomputed). So this seems ok. The raw correlations aren't that impressive, though Schrimpf et al assure us that this is quite good compared to the noise ceiling. And gpt2 has never seen a single human brain! We can see that the deeper into gpt2's layer stack that we go, the better the correlation score, with the best ones occuring somewhere in the middle. But I'm not so easily convinced. Remember, that gpt2 has still never seen a single real life brain.

One thing that is potentially missed is that these fmri signals might, to a certain degree, autocorrelate. For instance, the signal value at one time point might not be a particularly bad predictor of the next timepoint, just like your heart beat isn't going to suddenly double in the course of a second. We can try this out below for the fmri data.

```
import numpy as np
from scipy.stats import pearsonr
benchmark=benchmark_pool['Blank2014fROI-encoding']
data = np.array(benchmark._target_assembly)
corrs=[]
skip=1
for i in range(data.shape[1]):
    #lag the signal by 'skip' steps
    v=data[:,i]
    x=v[:-skip]
    y=v[skip:]
    cor = pearsonr(x,y)[0]
    corrs.append(cor)
print(np.mean(corrs))

---> 0.41318839674224245
```

And aha! They do. Given that, we might wonder if there is a way we could trick a LinearRegression on random(ish) data to predict the right signal. Well seemingly we can. Below, I initialize a random embedding at the beginning of each story and then proceed by injecting a small amount of noise at each time step. For each neuroid dimension, I then randomly split the data into 0.9/0.1 train/test and train a linear regression to predict the neuroids from the random(ish) embeddings. 

```
from sklearn.linear_model import LinearRegression

#get the story name for each sample
stimuli_ids = xx._target_assembly['stimulus_id'].values
stimuli_ids = [x.split('.')[0] for x in stim]

embed_dim=3000
embeds=[]
#make a random embedding
cur_embed = np.random.normal(0, 1, [embed_dim])
prev_name = 'A'
for stimuli_id in stimuli_ids:
    #if stimuli_id is a new story, create a new random embedding
    if prev_name != stimuli_id:
        cur_embed = np.random.normal(0, 1, [embed_dim])
    #otherwise, just add a (fairly large) amount of noise to the current embedding  
    else:
        cur_embed = cur_embed + np.random.normal(0, 0.1, [embed_dim])
    prev_name=s
    embeds.append(cur_embed)
    
X=np.array(embeds)
corrs=[]

#for each neuroid, do a random train test split
#build a linear regression between the semi random embeddings and the target
for i in range(data.shape[1]):
    Y = data[:, i]
    train_idx = np.random.random(len(X))<0.9
    train_x = X[train_idx]
    train_y = Y[train_idx].reshape(-1,1)
    test_x = X[~train_idx]
    test_y = Y [~train_idx].reshape(-1,1)
    m = LinearRegression().fit(train_x, train_y)
    pred = m.predict(test_x)
    cor,p = pearsonr(pred.flatten(), test_y.flatten())
    corrs.append(cor)

print('Mean correlation with simple train/test split', np.mean(corrs))

---> Mean correlation with simple train/test split 0.41665761324952694
```

So my dumb embedding scheme effectively captures all of the autocorrelation implicit in the signals, and does much better than gpt2. The key here is, I think, that the embedding dimension is large enough for the linear regression to totally overfit to each sample in the trainset. We can bake this into Shrimpf et al's code with a simple hack:


In [ ]:
import neural_nlp
from neural_nlp import score as score_function
from neural_nlp.models import model_pool
import numpy as np
model='gpt2-xl'
layers=None
subsample=None
import itertools
from neural_nlp.utils import ordered_set
from brainio.assemblies import DataAssembly, walk_coords, merge_data_arrays, array_is_element

def listen_to(candidate, stimulus_set, reset_column='story', average_sentence=True):
    """
    Code is from neural_nlp.benchmarks.neural.listen_to
    Add code to replace story_activations with our random trick embeddings of the same size.
    
    Pass a stimulus_set through a model candidate.
    Operates on a sentence-based stimulus_set.
    """
    activations = []
    for story in ordered_set(stimulus_set[reset_column].values):
        story_stimuli = stimulus_set[stimulus_set[reset_column] == story]

        story_stimuli.name = f"{stimulus_set.name}-{story}"
        story_activations = candidate(stimuli=story_stimuli, average_sentence=average_sentence)
        
        #This is the only addition to the code - add our random trick in place of
        #gpt2's activations
        embed=np.random.normal(0,1 , [story_activations.shape[1]])
        new_activ = [embed]
        for i in range(1,story_activations.shape[0]):
            embed = embed + np.random.normal(0, 0.001, story_activations.shape[1])
            new_activ.append(embed)
        story_activations[:] = new_activ
        #End addition.
        
        
        activations.append(story_activations)
        
    model_activations = merge_data_arrays(activations)
    # merging does not maintain stimulus order. the following orders again
    idx = [model_activations['stimulus_id'].values.tolist().index(stimulus_id) for stimulus_id in
           itertools.chain.from_iterable(s['stimulus_id'].values for s in activations)]
    assert len(set(idx)) == len(idx), "Found duplicate indices to order activations"
    model_activations = model_activations[{'presentation': idx}]
    #the model activations returned here are actually ordered by story
   
    return model_activations

neural_nlp.benchmarks.listen_to = listen_to

from neural_nlp.benchmarks.neural import Blank2014fROIEncoding


#need to change __call__ method to use the listen_to method above
class Blank2014Fixed(Blank2014fROIEncoding):
    
    def __call__(self, candidate):
#         _logger.info('Computing activations')
        model_activations = listen_to(candidate, self._target_assembly.attrs['stimulus_set'])
        assert set(model_activations['stimulus_id'].values) == set(self._target_assembly['stimulus_id'].values)
#         _logger.info('Scoring model')
        score = self.apply_metric(model_activations, self._target_assembly).copy()
        
        normalized_score,raw_score = self.ceiling_normalize(score)
    

        return raw_score, normalized_score
    
benchmark_pool['Blank2014Fixed1'] = Blank2014Fixed('Blank2014fROI-encoding')

model='gpt2-xl'
layers=None
subsample=None
bencmark='Blank2014Fixed1'

(raw, normalized )= score_function(model='gpt2-xl',
            layers=layers,
            subsample=subsample,
            benchmark=benchmark)
            
            
print([np.round(x,4) for x in normalized.values[:,0]])
---->[1.6771, 1.7417, 1.5778, 1.7718, 1.6413, 1.6526, 1.6702, 1.6343,
 1.6861, 1.6316, 1.6659, 1.621, 1.7315, 1.7783, 1.7054, 1.6932, 1.5978,
  1.7578, 1.6671, 1.7184, 1.6518, 1.6154, 1.6411, 1.7644, 1.5474, 1.5287, 
  ]1.6716, 1.7877, 1.7792, 1.707, 1.7296, 1.6836, 1.7145, 1.7302, 1.7645, 
  1.7717, 1.568, 1.5788, 1.7494, 1.7074, 1.7392, 1.7495, 1.6844,
1.7248, 1.6814, 1.6247, 1.6582, 1.6814, 1.7328]```


 The embeddings in the test set will be similar enough to their preceeding partners in the trainset, that in effect the signal autocorrelation can be learned from randomized embeddings. The discerning reader here will point out that I am using a time insensitive train/test split. Surely to god, the authors haven't done [that](https://github.com/brain-score/brain-score/blob/master/brainscore/metrics/transformations.py#L184)?

class Split:
    class Defaults:
        splits = 10
        train_size = .9
        split_coord = 'stimulus_id'
        stratification_coord = 'object_name'  # cross-validation across images, balancing objects
        unique_split_values = False
        random_state = 1

    def __init__(self,
                 splits=Defaults.splits, train_size=None, test_size=None,
                 split_coord=Defaults.split_coord, stratification_coord=Defaults.stratification_coord, kfold=False,
                 unique_split_values=Defaults.unique_split_values, random_state=Defaults.random_state):
        super().__init__()
        if train_size is None and test_size is None:
            train_size = self.Defaults.train_size
        if kfold:
            
            assert (train_size is None or train_size == self.Defaults.train_size) and test_size is None
            if stratification_coord:
             
                self._split = StratifiedKFold(n_splits=splits, shuffle=True, random_state=random_state)
            else:
              
                self._split = KFold(n_splits=splits, shuffle=True, random_state=random_state)
        else:
            if stratification_coord:
               
                self._split = StratifiedShuffleSplit(
                    n_splits=splits, train_size=train_size, test_size=test_size, random_state=random_state)
            else:
               
                
                self._split = ShuffleSplit(
                    n_splits=splits, train_size=train_size, test_size=test_size, random_state=random_state)
        self._split_coord = split_coord
        self._stratification_coord = stratification_coord
        self._unique_split_values = unique_split_values

        self._logger = logging.getLogger(fullname(self))

...
...
...
```

Oops? Although Schrimpf et al are careful enough to crossvalidate with sklearn `StratifiedKFold`, they have hardcoded `shuffle=True`![^2] Thus they are doing an indiscriminate, time insensitive train/test split! The voxel values from the end of each story are mixed up with those at the beginning, and the test values are just randomly missing points in the time series. So we could fix it like this

```
...
self._split = KFold(n_splits=splits, shuffle=False)
...
```

Then recomputing the results:

```
...
...
print(list([np.round(x,4) for x in raw.mean(axis=1).values]))
--->[0.0099, 0.0065, 0.003, -0.0029, 0.0054, 0.0101, 0.0051, 0.0047, -0.0082, 0.0025,
 0.0047, 0.0088, 0.008, 0.0067, 0.0053, 0.0134, 0.0086, 0.0041, 0.0133,
  0.0174, 0.0131, 0.0053, 0.0045, 0.0043, 0.0075, 0.0127, 0.0147, 0.0166,
   0.0142, 0.0169, 0.0151, 0.0124, 0.0179, 0.0143, 0.0076, 0.0002, -0.0007, 
   -0.0005, -0.0002, 0.0005, -0.0019, -0.0001, -0.0018, 0.0008, 0.0022, -0.0002,
    0.0029, 0.0018, 0.0042]

print([np.round(x,4) for x in normalized.values[:,0]])
--->[0.0486, 0.0586, 0.0251, -0.017, 0.075, 0.0555, 0.0691,
 0.0358, -0.041, 0.0167, 0.0174, 0.0582, 0.0196, 0.0136,
  0.0446, 0.0415, 0.0772, 0.063, 0.0271, 0.1104, 0.0601,
   0.013, 0.0308, -0.031, 0.0151, 0.0978, 0.0963, 0.1133,
    0.1328, 0.1853, 0.1706, 0.0971, 0.1552, 0.1403, 0.023, 
    0.0135, 0.0109, -0.0029, -0.0107, -0.0227, -0.0169,
     0.0412, 0.0203, 0.0424, -0.0042, -0.0253, 0.0212, 0.0083, 0.0184]
```

Those scores are really different. But then I think there might still be some chance of overlap here? If the signal is in essence, a time series, then surely no future data should going into the train set? Rather we should be splitting each story so that we train only on the values at the beginning/middle and test only on those at the end. E.g:

```
...
...
train_idx = []
test_idx=[]
from collections import defaultdict
d=defaultdict(list)
stimulus_ids = target_assembly['stimulus_id'].values
for idx, stimulus_id in enumerate(stimulus_ids):

    stimulus_id=stimulus_id.split('.')[0]
    d[stimulus_id].append(idx)

for stimulus_id in d:
    idxs=d[stimulus_id]
    q=int(len(idxs)*0.8)
    train_idx.extend(list(idxs[:q]))
    test_idx.extend(list(idxs[q:]))
...
...
```



```
print(list([np.round(x,4) for x in raw.mean(axis=1).values]))
--->[0.0022, 0.0179, 0.008, 0.0018, 0.0106, 0.019, 0.0127, 0.0126, 0.0002,
 0.0037, 0.0047, 0.0111, 0.0089, 0.0147, 0.0069, 0.0161, 0.0131, 0.0064, 
 0.0164, 0.0191, 0.015, 0.0075, 0.0083, 0.0086, 0.0142, 0.0172, 0.0207, 
 0.0222, 0.0207, 0.0226, 0.0204, 0.0202, 0.0269, 0.023, 0.0183, 0.0088, 
 0.0094, 0.0092, 0.0062, 0.0065, 0.0043, 0.0041, -0.0002, 0.0023, 0.0063, 
 0.0045, 0.005, 0.0034, 0.0083]

print([np.round(x,4) for x in normalized.values[:,0]])
--->[0.0268, -0.0866, 0.1223, -0.0465, -0.1973, 0.0199, 
-0.0719, 0.0215, -0.0109, -0.1299, 0.083, -0.0626, -0.0418, 
-0.0597, 0.0077, 0.0426, -0.0827, 0.0612, 0.0772, 0.1293, 
0.0654, -0.0816, 0.0846, -0.017, 0.1064, 0.0623, 0.0701, 
0.046, 0.1553, 0.1693, 0.0432, 0.1265, 0.008, 0.0444, 0.0082, 
-0.0185, -0.0137, -0.0178, 0.0317, 0.0407, -0.0013, 0.0121, 
-0.0307, 0.0045, 0.0989, -0.1037, -0.0928, -0.153, -0.0165]
```

As the gpt2 activations have been saved, it recomputes pretty quickly, and we get the above results. The brain scores have now pretty much collapsed. The mean normalized brain_score in the first instance was `0.21432`. For the KFold fix, it went down to `0.044738`. Now it's `0.008828`. And look at the raw correlation values! There are still one or two layers showing some layer/brain correlation, but the values are tiny, and we have had 49 shots at this so its quite likely just random. I don't know enough about neuroscience, so perhaps these are still significant correlations, but I would be inclined to call them coincidental. 


So does this paper demonstrate anything at all? To be fair, I have only examined one dataset, whereas Schrimpf et al provide four in their code. I couldn't get the loader for Federonko-2016 to work properly, and Perirea-2018 with its 150,000 neuroid values looked as if it would take several days for me to compute results for. I did find a lower autocorrelation value on the other datasets that I examined (e.g what...). Additionally, as noted below I couldn't get Schrimpf et al's specific install script to work, so there is some danger of package version mismatch. And of course, this doesn't say anything for the other recent papers. Cauchete et al, for instance, claim to be using a different kind of train/test split (GroupKFold?). I found some matlab code for [which paper], but I don't do matlab sorry :(. I guess we can just hope that all those other researchers are doing things better!

[^1]: I had some problems installing this. In the end, what seemed to work was installing nltk, torch and tensorflow separately with pip/conda, grabbing the latest [brainscore](https://github.com/brain-score/brain-score/tree/master/brainscore) and [brainio_base](https://github.com/brain-score/brainio_base) packages directly from Schrimpf's github, and only then running the neural_nlp setup.py script.
[^2]: [https://github.com/brain-score/brain-score/blob/master/brainscore/metrics/transformations.py#L184](https://github.com/brain-score/brain-score/blob/master/brainscore/metrics/transformations.py#L184) This line in the brainscore code predates the release of the paper by at least 2 years.
